# Pre-Processing Techniques

## Imports

In [2]:
import pandas as pd

## Read the Relevant Dataframes and Merge

In [31]:
# Read the dfs
ptau = pd.read_csv('Data/Plasma/ADNI_BLENNOWPLASMANFL_10_03_18_27Nov2023.csv')
nfl = pd.read_csv('Data/Plasma/BLENNOWPLASMATAU_07Feb2024.csv')
ab = pd.read_csv('Data/Plasma/UPENNPLASMA_27Nov2023.csv')

# Confirm Shapes before merging
print("ptau shape is", ptau.shape)
print("nfl shape is", nfl.shape)
print("ab shape is", ab.shape)

df = pd.merge(ptau, nfl, on=['RID', 'VISCODE'], how='inner')[['RID', 'VISCODE', 'PLASMA_NFL', 'PLASMATAU']]
print("\nBLEN Data Makes the Shape", df.shape)

# Merge UPENN in
df = pd.merge(df, ab, on=['RID', 'VISCODE'], how='inner').drop("update_stamp", axis=1)
df

# NEXT STEPS: 
# - Get Classifications
# - Visualise the data on Graphs
# - Try with other sets


ptau shape is (598, 9)
nfl shape is (581, 8)
ab shape is (2453, 5)

BLEN Data Makes the Shape (580, 4)


,RID,VISCODE,PLASMA_NFL,PLASMATAU,AB40,AB42
0,2,bl,28.2,2.41,153.7,50.9
1,3,bl,188.3,3.84,263.3,58.1
2,4,bl,31.2,3.76,192.0,59.8
3,5,bl,36.3,3.81,246.1,46.7
4,6,bl,52.5,2.86,289.0,44.3
...,...,...,...,...,...,...
519,1402,bl,94.1,4.45,143.6,37.4
520,1409,bl,46.4,3.54,111.5,25.8
521,1414,bl,18.7,1.45,170.5,22.2
522,1419,bl,25.8,1.56,206.7,40.1
